In [1]:
spark

In [2]:
import pandas as pd

In [4]:
df = spark.read.csv("/data/bicy/*", encoding='cp949', inferSchema = True, header=True)

In [5]:
df.count()

22334740

In [6]:
df.printSchema()

root
 |-- 자전거번호: string (nullable = true)
 |-- 대여일시: timestamp (nullable = true)
 |-- 대여 대여소번호: integer (nullable = true)
 |-- 대여 대여소명: string (nullable = true)
 |-- 대여거치대: integer (nullable = true)
 |-- 반납일시: timestamp (nullable = true)
 |-- 반납대여소번호: string (nullable = true)
 |-- 반납대여소명: string (nullable = true)
 |-- 반납거치대: string (nullable = true)
 |-- 이용시간(분): integer (nullable = true)
 |-- 이용거리(M): double (nullable = true)
 |-- 생년: string (nullable = true)
 |-- 성별: string (nullable = true)
 |-- 이용자종류: string (nullable = true)
 |-- 대여대여소ID: string (nullable = true)
 |-- 반납대여소ID: string (nullable = true)



In [7]:
df.first()

Row(자전거번호='SPB-39194', 대여일시=datetime.datetime(2023, 1, 1, 0, 2, 13), 대여 대여소번호=1554, 대여 대여소명='번동사거리', 대여거치대=0, 반납일시=datetime.datetime(2023, 1, 1, 0, 2, 43), 반납대여소번호='01554', 반납대여소명='번동사거리', 반납거치대='0', 이용시간(분)=0, 이용거리(M)=0.0, 생년='1977', 성별='₩N', 이용자종류='내국인', 대여대여소ID='ST-2127', 반납대여소ID='ST-2127')

In [10]:
from pyspark.sql.functions import *
df.select(col("자전거번호"), col("성별")).show()

+----------+----+
|자전거번호|성별|
+----------+----+
| SPB-39194|  ₩N|
| SPB-43457|  ₩N|
| SPB-44383|   M|
| SPB-37186|   M|
| SPB-59440|   M|
| SPB-35099|   M|
| SPB-58162|  ₩N|
| SPB-48823|  ₩N|
| SPB-59358|   M|
| SPB-51038|   M|
| SPB-64954|   M|
| SPB-59872|  ₩N|
| SPB-58084|  ₩N|
| SPB-53276|  ₩N|
| SPB-31838|  ₩N|
| SPB-33542|   M|
| SPB-34326|   F|
| SPB-60653|  ₩N|
| SPB-33248|   F|
| SPB-42593|   M|
+----------+----+
only showing top 20 rows



In [14]:
df.select(expr(" '자전거번호' as bicycle_num ")).head(3)

[Row(bicycle_num='자전거번호'), Row(bicycle_num='자전거번호'), Row(bicycle_num='자전거번호')]

In [15]:
df.select('*', expr("('자전거번호' = SPB-39194) as '자전거'")).show()

ParseException: 
Syntax error at or near ''자전거'': extra input ''자전거''(line 1, pos 25)

== SQL ==
('자전거번호' = SPB-39194) as '자전거'
-------------------------^^^


In [16]:
df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull(), c 
                           )).alias(c)
                    for c in df.columns])


In [17]:
df2.show()

+----------+--------+---------------+-------------+----------+--------+--------------+------------+----------+------------+-----------+----+------+----------+------------+------------+
|자전거번호|대여일시|대여 대여소번호|대여 대여소명|대여거치대|반납일시|반납대여소번호|반납대여소명|반납거치대|이용시간(분)|이용거리(M)|생년|  성별|이용자종류|대여대여소ID|반납대여소ID|
+----------+--------+---------------+-------------+----------+--------+--------------+------------+----------+------------+-----------+----+------+----------+------------+------------+
|         0|       0|              0|            0|         0|       0|             0|           0|         0|           0|          0|   1|695267|         0|           0|           0|
+----------+--------+---------------+-------------+----------+--------+--------------+------------+----------+------------+-----------+----+------+----------+------------+------------+



In [18]:
df.groupby("성별").agg( count("성별").alias("값")).show()

+----+-------+
|성별|     값|
+----+-------+
|   F|5917315|
|   m|   3295|
|null|      0|
|   f|   1265|
|   M|9392045|
|  ₩N|6325553|
+----+-------+



In [19]:
df.groupby("성별").agg( avg("이용시간(분)").alias("값")).show()

+----+------------------+
|성별|                값|
+----+------------------+
|   F| 22.06580653556554|
|   m|17.090136570561455|
|null|20.063203057242756|
|   f|16.621343873517787|
|   M| 20.02762561295224|
|  ₩N| 21.02134548552514|
+----+------------------+



In [ ]:
df.withColumn("numberOne", lit(1)).show(2)

In [21]:
df = df.withColumn('성별', when(col('성별') == 'm', 'M').otherwise(col('성별')))

In [22]:
df.groupby("성별").agg( count("성별").alias("값")).show()

+----+-------+
|성별|     값|
+----+-------+
|   F|5917315|
|null|      0|
|   f|   1265|
|   M|9395340|
|  ₩N|6325553|
+----+-------+



In [24]:
df = df.withColumn('성별', when(col('성별') == 'f', 'F').otherwise(col('성별')))
# f라면 F로 바꾸고 그렇지 않다면 원래 값 그대로 유지 

In [26]:
df.groupby("성별").agg(count("성별").alias("값")).show()

+----+-------+
|성별|     값|
+----+-------+
|   F|5918580|
|null|      0|
|   M|9395340|
|  ₩N|6325553|
+----+-------+

